# RO System Simulation with MCAS Property Package and Recycle Support

This notebook template combines the MCAS (Multi-Component Aqueous Solution) property package
with recycle stream support for high recovery RO systems. It provides:
- Ion-specific modeling and rejection calculations
- Recycle stream implementation with proper mass balance
- Pump pressure optimization to meet target recoveries
- Scaling prediction and ion accumulation tracking

In [ ]:
# Parameters cell - will be replaced by papermill
project_root = "/path/to/project"  # Will be replaced by papermill
configuration = {}
feed_salinity_ppm = 5000
feed_temperature_c = 25.0
membrane_type = "brackish"
membrane_properties = None
optimize_pumps = True
# Ion composition in mg/L
feed_ion_composition = None
# Initialization strategy
initialization_strategy = "sequential"

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from pyomo.environ import *
from pyomo.network import Arc
import sys
import os
import platform

# Add parent directory to path for utils imports using project_root parameter
sys.path.insert(0, project_root)

# Ensure PyNumero DLL can be found on Windows
if platform.system() == "Windows":
    pyomo_lib_path = os.path.join(os.path.expanduser("~"), "AppData", "Local", "Pyomo", "lib")
    if os.path.exists(pyomo_lib_path):
        if 'PATH' in os.environ:
            if pyomo_lib_path not in os.environ['PATH']:
                os.environ['PATH'] = f"{pyomo_lib_path};{os.environ['PATH']}"
        else:
            os.environ['PATH'] = pyomo_lib_path
        print(f"Added Pyomo lib path to PATH: {pyomo_lib_path}")

# Import our MCAS builder
from utils.mcas_builder import (
    build_mcas_property_configuration,
    check_electroneutrality,
    get_total_dissolved_solids,
    calculate_ionic_strength
)

# Import our elegant initialization utilities
from utils.ro_initialization import (
    calculate_required_pressure,
    initialize_pump_with_pressure,
    initialize_ro_unit_elegant,
    initialize_multistage_ro_elegant
)

# WaterTAP imports
from watertap.core.solvers import get_solver
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType
)
from watertap.unit_models.pressure_changer import Pump
from watertap.property_models.multicomp_aq_sol_prop_pack import MCASParameterBlock

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.models.unit_models import Feed, Product, Mixer, Separator
from idaes.models.unit_models.mixer import MixingType, MomentumMixingType

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(level=logging.INFO)

# Results storage
results = {}

## Check Configuration for Recycle

In [ ]:
# Check if recycle is configured - fixing nested structure access
recycle_info = configuration.get('recycle_info', {})
has_recycle = recycle_info.get('uses_recycle', False)
recycle_ratio = recycle_info.get('recycle_ratio', 0)
recycle_split_ratio = recycle_info.get('recycle_split_ratio', 0)
recycle_flow_m3h = recycle_info.get('recycle_flow_m3h', 0)
effective_feed_flow_m3h = recycle_info.get('effective_feed_flow_m3h', configuration.get('feed_flow_m3h', 0))

if has_recycle:
    print(f"Recycle configuration detected:")
    print(f"  Recycle ratio: {recycle_ratio*100:.1f}%")
    print(f"  Recycle flow: {recycle_flow_m3h:.1f} m³/h")
    print(f"  Recycle split ratio: {recycle_split_ratio*100:.1f}%")
    print(f"  Fresh feed flow: {configuration.get('feed_flow_m3h', 0):.1f} m³/h")
    print(f"  Effective feed flow: {effective_feed_flow_m3h:.1f} m³/h")
    print(f"  Effective feed salinity will be calculated based on ion accumulation")
else:
    print("No recycle configuration detected. Building standard RO model with MCAS.")
    print(f"  Feed flow: {configuration.get('feed_flow_m3h', 0):.1f} m³/h")
    print(f"  Feed salinity: {feed_salinity_ppm:.0f} ppm")

## Define Feed Ion Composition

In [ ]:
# Define typical ion compositions for different water types
TYPICAL_COMPOSITIONS = {
    "brackish": {
        "Na+": 1200,
        "Ca2+": 120,
        "Mg2+": 60,
        "K+": 20,
        "Cl-": 2100,
        "SO4-2": 200,
        "HCO3-": 150,
        "SiO3-2": 10
    },
    "seawater": {
        "Na+": 10800,
        "Ca2+": 420,
        "Mg2+": 1300,
        "K+": 400,
        "Sr2+": 8,
        "Cl-": 19400,
        "SO4-2": 2700,
        "HCO3-": 140,
        "Br-": 70,
        "F-": 1.3
    }
}

# Use provided composition or typical based on salinity
if feed_ion_composition is None:
    # Scale typical composition to match target salinity
    typical = TYPICAL_COMPOSITIONS[membrane_type]
    typical_tds = sum(typical.values())
    scale_factor = feed_salinity_ppm / typical_tds
    
    feed_ion_composition = {
        ion: conc * scale_factor 
        for ion, conc in typical.items()
    }
else:
    # Parse JSON string if provided
    import json
    if isinstance(feed_ion_composition, str):
        feed_ion_composition = json.loads(feed_ion_composition)

# Check and report composition
print(f"Fresh feed water composition (mg/L):")
for ion, conc in sorted(feed_ion_composition.items()):
    print(f"  {ion:8s}: {conc:8.1f}")

actual_tds = get_total_dissolved_solids(feed_ion_composition)
print(f"\nTotal TDS: {actual_tds:.0f} mg/L")

# Check electroneutrality
is_neutral, imbalance = check_electroneutrality(feed_ion_composition)
if not is_neutral:
    print(f"\nWarning: Charge imbalance of {imbalance:.1%}")

# Calculate ionic strength
ionic_strength = calculate_ionic_strength(feed_ion_composition)
print(f"Ionic strength: {ionic_strength:.3f} mol/L")

if has_recycle:
    print(f"\nNote: Effective feed composition will be higher due to recycle.")
    print(f"Ion accumulation factors will be calculated during model building.")

## Build MCAS Property Configuration

In [ ]:
# Build MCAS property configuration
mcas_config = build_mcas_property_configuration(
    feed_composition=feed_ion_composition,
    include_scaling_ions=True,
    include_ph_species=True
)

print("MCAS Configuration created with components:")
print(f"  Solute list: {', '.join(mcas_config['solute_list'])}")
print(f"  Activity model: {mcas_config['activity_coefficient_model']}")
print(f"  Scaling ions tracked: {len(mcas_config.get('scaling_ions', {}))} types")

## Build WaterTAP Model with MCAS and Recycle Support

In [ ]:
def build_ro_model_mcas_with_recycle(config_data, mcas_config, feed_salinity_ppm, 
                                      feed_temperature_c, membrane_type, has_recycle):
    """
    Build WaterTAP RO model with MCAS property package and optional recycle support.
    
    Key features:
    - Uses MCASParameterBlock for ion-specific modeling
    - Implements recycle loop with Mixer and Separator if configured
    - Uses consistent arc naming to avoid initialization issues
    - Sets up model structure before fixing membrane properties
    - Uses SKK transport model for better field data matching
    """
    # Create concrete model
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    
    # Import MaterialFlowBasis and TransportModel
    from watertap.property_models.multicomp_aq_sol_prop_pack import MaterialFlowBasis
    from watertap.core.membrane_channel_base import TransportModel
    
    # Filter MCAS config to only include parameters MCASParameterBlock accepts
    mcas_params = {
        'solute_list': mcas_config['solute_list'],
        'mw_data': mcas_config['mw_data'],
        'material_flow_basis': MaterialFlowBasis.mass,  # Use mass flow for RO
    }
    
    # Add optional parameters if they exist
    optional_params = [
        'stokes_radius_data', 'diffusivity_data', 'charge', 
        'activity_coefficient_model'
    ]
    
    for param in optional_params:
        if param in mcas_config:
            mcas_params[param] = mcas_config[param]
    
    # Create MCAS property package
    m.fs.properties = MCASParameterBlock(**mcas_params)
    
    # Feed conditions
    fresh_feed_flow_m3_s = config_data['feed_flow_m3h'] / 3600  # Convert to m³/s
    
    # Create fresh feed unit
    m.fs.fresh_feed = Feed(property_package=m.fs.properties)
    
    if has_recycle:
        # Create recycle components
        print("Building model with recycle support...")
        
        # Separator for final concentrate split
        m.fs.recycle_split = Separator(
            property_package=m.fs.properties,
            outlet_list=["disposal", "recycle"]
        )
        
        # Mixer for fresh feed and recycle
        m.fs.feed_mixer = Mixer(
            property_package=m.fs.properties,
            inlet_list=["fresh", "recycle"],
            energy_mixing_type=MixingType.none,  # Required for MCAS
            momentum_mixing_type=MomentumMixingType.none
        )
        
        # Effective feed is mixer outlet
        effective_feed = m.fs.feed_mixer.outlet
    else:
        # No recycle - fresh feed goes directly to RO system
        effective_feed = m.fs.fresh_feed.outlet
    
    # Build stages - using n_stages consistently
    n_stages = config_data.get('n_stages', config_data.get('stage_count', 1))
    
    # Create all RO stages and pumps with consistent naming
    for i in range(1, n_stages + 1):
        # Create pump for stage
        setattr(m.fs, f"pump{i}", Pump(property_package=m.fs.properties))
        
        # Create RO stage with SKK transport model
        setattr(m.fs, f"ro_stage{i}", ReverseOsmosis0D(
            property_package=m.fs.properties,
            has_pressure_change=True,
            concentration_polarization_type=ConcentrationPolarizationType.calculated,
            mass_transfer_coefficient=MassTransferCoefficient.calculated,
            pressure_change_type=PressureChangeType.fixed_per_stage,
            transport_model=TransportModel.SKK  # Use SKK model
        ))
        
        # Create permeate product for each stage
        setattr(m.fs, f"stage_product{i}", Product(property_package=m.fs.properties))
    
    # Create final products
    if has_recycle:
        m.fs.disposal_product = Product(property_package=m.fs.properties)
    else:
        m.fs.concentrate_product = Product(property_package=m.fs.properties)
    
    # Build connectivity with consistent naming
    if has_recycle:
        # Fresh feed to mixer
        m.fs.fresh_to_mixer = Arc(
            source=m.fs.fresh_feed.outlet,
            destination=m.fs.feed_mixer.fresh
        )
        
        # Mixer to first pump
        m.fs.mixer_to_pump1 = Arc(
            source=m.fs.feed_mixer.outlet,
            destination=m.fs.pump1.inlet
        )
    else:
        # Fresh feed directly to first pump
        m.fs.feed_to_pump1 = Arc(
            source=m.fs.fresh_feed.outlet,
            destination=m.fs.pump1.inlet
        )
    
    # Connect first pump to first RO
    m.fs.pump1_to_ro_stage1 = Arc(
        source=m.fs.pump1.outlet,
        destination=m.fs.ro_stage1.inlet
    )
    
    # Connect permeate from first RO to product
    m.fs.ro_stage1_perm_to_prod = Arc(
        source=m.fs.ro_stage1.permeate,
        destination=m.fs.stage_product1.inlet
    )
    
    # Connect stages if multiple
    if n_stages > 1:
        for i in range(1, n_stages):
            # Concentrate of stage i to pump i+1
            setattr(m.fs, f"ro_stage{i}_to_pump{i+1}", Arc(
                source=getattr(m.fs, f"ro_stage{i}").retentate,
                destination=getattr(m.fs, f"pump{i+1}").inlet
            ))
            
            # Pump i+1 to RO i+1
            setattr(m.fs, f"pump{i+1}_to_ro_stage{i+1}", Arc(
                source=getattr(m.fs, f"pump{i+1}").outlet,
                destination=getattr(m.fs, f"ro_stage{i+1}").inlet
            ))
            
            # Permeate to product
            setattr(m.fs, f"ro_stage{i+1}_perm_to_prod{i+1}", Arc(
                source=getattr(m.fs, f"ro_stage{i+1}").permeate,
                destination=getattr(m.fs, f"stage_product{i+1}").inlet
            ))
    
    # Connect final concentrate
    final_stage = n_stages
    if has_recycle:
        # Final concentrate to recycle split
        m.fs.final_conc_to_split = Arc(
            source=getattr(m.fs, f"ro_stage{final_stage}").retentate,
            destination=m.fs.recycle_split.inlet
        )
        
        # Recycle split outputs
        m.fs.split_to_disposal = Arc(
            source=m.fs.recycle_split.disposal,
            destination=m.fs.disposal_product.inlet
        )
        
        m.fs.split_to_recycle = Arc(
            source=m.fs.recycle_split.recycle,
            destination=m.fs.feed_mixer.recycle
        )
    else:
        # Final concentrate to product
        m.fs.final_conc_arc = Arc(
            source=getattr(m.fs, f"ro_stage{final_stage}").retentate,
            destination=m.fs.concentrate_product.inlet
        )
    
    # Apply arcs to expand the network
    TransformationFactory("network.expand_arcs").apply_to(m)
    
    # NOW set membrane properties after model structure is built
    for i in range(1, n_stages + 1):
        stage_data = config_data['stages'][i-1]
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Set reflection coefficient for SKK model
        if membrane_type == "seawater":
            ro.reflect_coeff.fix(0.98)  # Higher rejection for seawater
        else:  # brackish
            ro.reflect_coeff.fix(0.95)  # Standard rejection for brackish
        
        # Membrane properties based on type
        if membrane_type == "seawater":
            ro.A_comp.fix(1.5e-12)  # m/s/Pa
            # Set B values for each ion
            for comp in mcas_config['solute_list']:
                if comp in ['Na_+', 'Cl_-']:
                    ro.B_comp[0, comp].fix(1.0e-8)  # m/s
                elif comp in ['Ca_2+', 'Mg_2+', 'SO4_2-']:
                    ro.B_comp[0, comp].fix(5.0e-9)  # m/s - higher rejection
                else:
                    ro.B_comp[0, comp].fix(8.0e-9)  # m/s
        else:  # brackish
            ro.A_comp.fix(4.2e-12)  # m/s/Pa
            for comp in mcas_config['solute_list']:
                if comp in ['Na_+', 'Cl_-']:
                    ro.B_comp[0, comp].fix(3.5e-8)  # m/s
                elif comp in ['Ca_2+', 'Mg_2+', 'SO4_2-']:
                    ro.B_comp[0, comp].fix(1.5e-8)  # m/s
                else:
                    ro.B_comp[0, comp].fix(2.5e-8)  # m/s
        
        # Set fixed pressure drop
        ro.deltaP.fix(-0.5e5)  # Pa
        
        # Fix permeate pressure
        ro.permeate.pressure[0].fix(101325)  # 1 atm
        
        # Channel geometry
        ro.feed_side.channel_height.fix(0.001)  # 1 mm
        ro.feed_side.spacer_porosity.fix(0.85)
        
        # Set membrane area and width
        required_area = stage_data.get('membrane_area_m2', 
                                     stage_data.get('area_m2', 260.16))
        ro.area.fix(required_area)
        
        # Fix width based on vessel configuration
        n_vessels = stage_data.get('n_vessels', stage_data.get('vessel_count', 1))
        aggregate_width = n_vessels * 5.0  # 5m effective width per vessel
        ro.width.fix(aggregate_width)
    
    # Set fresh feed conditions (always based on fresh feed, not effective)
    feed_state = m.fs.fresh_feed.outlet
    
    # Temperature and pressure
    feed_state.temperature.fix(273.15 + feed_temperature_c)
    feed_state.pressure.fix(101325)  # 1 atm
    
    # Component flows based on ion composition (mass basis)
    ion_composition_mg_l = mcas_config['ion_composition_mg_l']
    total_ion_flow_kg_s = 0
    
    # Set ion flows
    for comp in mcas_config['solute_list']:
        conc_mg_l = ion_composition_mg_l[comp]
        ion_flow_kg_s = conc_mg_l * fresh_feed_flow_m3_s / 1000  # mg/L * m³/s / 1000 = kg/s
        feed_state.flow_mass_phase_comp[0, 'Liq', comp].fix(ion_flow_kg_s)
        total_ion_flow_kg_s += ion_flow_kg_s
    
    # Water flow
    water_flow_kg_s = fresh_feed_flow_m3_s * 1000 - total_ion_flow_kg_s  # ~1000 kg/m³
    feed_state.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(water_flow_kg_s)
    
    # Set recycle split if applicable
    if has_recycle:
        # Get split ratio from configuration
        recycle_split_ratio = recycle_info.get('recycle_split_ratio', 0.5)
        m.fs.recycle_split.split_fraction[0, "recycle"].fix(recycle_split_ratio)
        m.fs.recycle_split.split_fraction[0, "disposal"].fix(1 - recycle_split_ratio)
    
    # Set pump efficiencies
    for i in range(1, n_stages + 1):
        getattr(m.fs, f"pump{i}").efficiency_pump.fix(0.8)
    
    # Set scaling factors
    m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
    for comp in mcas_config['solute_list']:
        m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e4, index=("Liq", comp))
    calculate_scaling_factors(m)
    
    return m

In [ ]:
def initialize_and_solve_mcas(model, config_data, optimize_pumps=True):
    """
    Initialize and solve RO model with MCAS property package and recycle.
    
    This function properly handles pump optimization by:
    1. First initializing with fixed pump pressures for stability
    2. Then unfixing pumps and adding recovery constraints if optimize_pumps=True
    """
    m = model
    n_stages = config_data.get('n_stages', config_data.get('stage_count', 1))
    
    # Check for recycle
    recycle_info = config_data.get('recycle_info', {})
    has_recycle = recycle_info.get('uses_recycle', False)
    recycle_ratio = recycle_info.get('recycle_ratio', 0)
    
    logger = logging.getLogger(__name__)
    logger.info("=== Starting MCAS Recycle Initialization ===")
    logger.info(f"Number of stages: {n_stages}")
    logger.info(f"Has recycle: {has_recycle}")
    logger.info(f"Recycle ratio: {recycle_ratio}")
    logger.info(f"Optimize pumps: {optimize_pumps}")
    
    # Initialize fresh feed
    m.fs.fresh_feed.initialize()
    logger.info("Fresh feed initialized")
    
    # Initialize recycle system first (if present)
    if has_recycle:
        logger.info("\n=== Initializing Recycle System ===")
        
        # Start with zero recycle for initial solution
        logger.info("Phase 1: Initializing with zero recycle")
        
        # Temporarily set recycle to zero
        m.fs.recycle_split.split_fraction[0, "recycle"].fix(0)
        m.fs.recycle_split.split_fraction[0, "disposal"].fix(1)
        
        # Initialize feed mixer with only fresh feed
        # First propagate fresh feed to mixer
        propagate_state(arc=m.fs.fresh_to_mixer)
        
        # Initialize mixer with fixed states to start
        m.fs.feed_mixer.mixed_state[0].flow_mass_phase_comp.fix()
        m.fs.feed_mixer.mixed_state[0].temperature.fix(298.15)
        m.fs.feed_mixer.mixed_state[0].pressure.fix(101325)
        
        m.fs.feed_mixer.initialize()
        
        # Propagate from mixer
        propagate_state(arc=m.fs.mixer_to_pump1)
    else:
        # No recycle - propagate directly from feed
        propagate_state(arc=m.fs.feed_to_pump1)
    
    # Get feed TDS for pressure calculations
    feed_flows = {}
    for comp in m.fs.properties.solute_set | {'H2O'}:
        feed_flows[comp] = value(m.fs.fresh_feed.outlet.flow_mass_phase_comp[0, 'Liq', comp])
    
    h2o_flow = feed_flows['H2O']
    tds_flow = sum(v for k, v in feed_flows.items() if k != 'H2O')
    feed_tds_ppm = (tds_flow / (h2o_flow + tds_flow)) * 1e6
    
    logger.info(f"Feed TDS: {feed_tds_ppm:.0f} ppm")
    
    # Initialize stages with elegant initialization
    logger.info("\n=== Initializing RO Stages ===")
    
    current_tds_ppm = feed_tds_ppm
    for i in range(1, n_stages + 1):
        logger.info(f"\n--- Stage {i} ---")
        
        pump = getattr(m.fs, f"pump{i}")
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Get stage recovery target
        stage_data = config_data['stages'][i-1]
        target_recovery = stage_data.get('stage_recovery', 0.5)
        
        # Propagate to pump (already done for stage 1)
        if i > 1:
            propagate_state(arc=getattr(m.fs, f"ro_stage{i-1}_to_pump{i}"))
        
        # Calculate required pressure
        min_driving = 15e5 if target_recovery < 0.5 else 20e5
        if target_recovery > 0.7:
            min_driving = 25e5
        
        required_pressure = calculate_required_pressure(
            current_tds_ppm,
            target_recovery,
            permeate_pressure=101325,
            min_driving_pressure=min_driving,
            pressure_drop=0.5e5
        )
        
        # Add safety factor
        safety_factor = 1.1 + 0.1 * (i - 1) + 0.2 * max(0, target_recovery - 0.5)
        required_pressure = min(required_pressure * safety_factor, 80e5)
        
        # Initialize pump with fixed pressure
        initialize_pump_with_pressure(pump, required_pressure)
        
        # Propagate to RO
        propagate_state(arc=getattr(m.fs, f"pump{i}_to_ro_stage{i}"))
        
        # Initialize RO with elegant approach
        initialize_ro_unit_elegant(ro, target_recovery, verbose=True)
        
        # Update TDS for next stage
        current_tds_ppm = calculate_concentrate_tds(current_tds_ppm, target_recovery)
        
        # Propagate permeate to product
        propagate_state(arc=getattr(m.fs, f"ro_stage{i}_perm_to_prod{i if i > 1 else ''}"))
        getattr(m.fs, f"stage_product{i}").initialize()
    
    # Complete initialization of recycle components if present
    if has_recycle:
        # Initialize recycle splitter and disposal
        final_stage = n_stages
        propagate_state(arc=m.fs.final_conc_to_split)
        m.fs.recycle_split.initialize()
        
        propagate_state(arc=m.fs.split_to_disposal)
        m.fs.disposal_product.initialize()
        
        # Now set actual recycle ratio
        recycle_split_ratio = recycle_info.get('recycle_split_ratio', 0.5)
        logger.info(f"\nPhase 2: Setting recycle split ratio to {recycle_split_ratio}")
        m.fs.recycle_split.split_fraction[0, "recycle"].fix(recycle_split_ratio)
        m.fs.recycle_split.split_fraction[0, "disposal"].fix(1 - recycle_split_ratio)
        
        # Re-initialize splitter
        m.fs.recycle_split.initialize()
        
        # Unfix mixer states for recycle convergence
        m.fs.feed_mixer.mixed_state[0].flow_mass_phase_comp.unfix()
        m.fs.feed_mixer.mixed_state[0].temperature.unfix()
        m.fs.feed_mixer.mixed_state[0].pressure.unfix()
    else:
        # Initialize final concentrate product
        final_stage = n_stages
        propagate_state(arc=m.fs.final_conc_arc)
        m.fs.concentrate_product.initialize()
    
    # Check initial solution
    logger.info("\n=== Checking Initial Solution ===")
    for i in range(1, n_stages + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        h2o_in = value(ro.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        h2o_perm = value(ro.permeate.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        recovery = h2o_perm / h2o_in if h2o_in > 0 else 0
        logger.info(f"Stage {i} initial recovery: {recovery:.3f}")
    
    # If optimize_pumps, unfix pumps and add recovery constraints
    if optimize_pumps:
        logger.info("\n=== Setting up Pump Optimization ===")
        
        # First verify we have a feasible initial solution
        # Get solver (no parameters to get_solver)
        solver = get_solver()
        
        logger.info("Verifying initial solution...")
        results = solver.solve(m, tee=False, options={'linear_solver': 'ma27'})
        
        if results.solver.termination_condition != TerminationCondition.optimal:
            logger.warning(f"Initial solution not optimal: {results.solver.termination_condition}")
            logger.warning("Proceeding with pump optimization anyway...")
        
        # Now unfix pumps and add recovery constraints
        for i in range(1, n_stages + 1):
            pump = getattr(m.fs, f"pump{i}")
            ro = getattr(m.fs, f"ro_stage{i}")
            stage_data = config_data['stages'][i-1]
            target_recovery = stage_data.get('stage_recovery', 0.5)
            
            # Unfix pump pressure
            pump.outlet.pressure[0].unfix()
            logger.info(f"Stage {i}: Unfixed pump pressure (was {value(pump.outlet.pressure[0])/1e5:.1f} bar)")
            
            # Add recovery constraint
            constraint_name = f"recovery_constraint_stage{i}"
            setattr(m.fs, constraint_name,
                    Constraint(expr=ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'] == target_recovery))
            logger.info(f"Stage {i}: Added recovery constraint for {target_recovery:.3f}")
        
        # Check degrees of freedom
        dof = degrees_of_freedom(m)
        logger.info(f"\nDegrees of freedom after adding constraints: {dof}")
        
        if dof != 0:
            logger.warning(f"Expected 0 degrees of freedom, got {dof}")
    
    # Solve the model
    logger.info("\n=== Solving Model ===")
    # Get solver (no parameters to get_solver)
    solver = get_solver()
    
    if has_recycle and optimize_pumps:
        # Use successive substitution for recycle with pump optimization
        logger.info("Using successive substitution for recycle convergence")
        
        max_iter = 20
        tol = 1e-5
        
        for iteration in range(max_iter):
            # Store previous mixed flow
            prev_flow = value(m.fs.feed_mixer.mixed_state[0].flow_mass_phase_comp['Liq', 'H2O'])
            
            # Solve model
            results = solver.solve(m, tee=False, options={'linear_solver': 'ma27'})
            
            if results.solver.termination_condition != TerminationCondition.optimal:
                logger.error(f"Iteration {iteration+1}: Solver failed - {results.solver.termination_condition}")
                raise RuntimeError(f"Solver failed during recycle iteration {iteration+1}")
            
            # Check convergence
            curr_flow = value(m.fs.feed_mixer.mixed_state[0].flow_mass_phase_comp['Liq', 'H2O'])
            rel_change = abs(curr_flow - prev_flow) / prev_flow if prev_flow > 0 else 1
            
            logger.info(f"Iteration {iteration+1}: Mixed flow = {curr_flow:.4f} kg/s, relative change = {rel_change:.2e}")
            
            if rel_change < tol:
                logger.info(f"Converged after {iteration+1} iterations")
                break
        else:
            logger.warning(f"Did not converge after {max_iter} iterations")
    else:
        # Single solve for non-recycle or fixed pump cases
        results = solver.solve(m, tee=True, options={'linear_solver': 'ma27'})
        
        if results.solver.termination_condition != TerminationCondition.optimal:
            logger.error(f"Solver failed: {results.solver.termination_condition}")
            raise RuntimeError(f"Solver failed: {results.solver.termination_condition}")
    
    logger.info("\n=== Solution Complete ===")
    
    # Report final recoveries and pressures
    for i in range(1, n_stages + 1):
        pump = getattr(m.fs, f"pump{i}")
        ro = getattr(m.fs, f"ro_stage{i}")
        
        pressure = value(pump.outlet.pressure[0]) / 1e5  # bar
        recovery = value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'])
        
        logger.info(f"Stage {i}: Pressure = {pressure:.1f} bar, Recovery = {recovery:.3f}")
    
    return m

In [ ]:
def extract_results_mcas(model, config_data):
    """
    Extract comprehensive results from solved MCAS RO model with recycle support.
    
    Tracks ion-specific rejections, scaling potential, and mass balance.
    """
    m = model
    n_stages = config_data.get('n_stages', config_data.get('stage_count', 1))
    
    # Check for recycle
    recycle_info = config_data.get('recycle_info', {})
    has_recycle = recycle_info.get('uses_recycle', False)
    
    results = {
        "status": "success",
        "configuration": config_data,
        "has_recycle": has_recycle,
        "performance": {},
        "stage_results": [],
        "ion_tracking": {},
        "mass_balance": {},
        "economics": {}
    }
    
    # Extract stage-wise results
    total_perm_flow = 0
    total_power = 0
    
    for i in range(1, n_stages + 1):
        pump = getattr(m.fs, f"pump{i}")
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Flow rates
        feed_h2o = value(ro.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        perm_h2o = value(ro.permeate.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        conc_h2o = value(ro.retentate.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        
        # Recovery
        recovery = perm_h2o / feed_h2o if feed_h2o > 0 else 0
        
        # Pressures
        feed_pressure = value(ro.inlet.pressure[0]) / 1e5
        conc_pressure = value(ro.retentate.pressure[0]) / 1e5
        perm_pressure = value(ro.permeate.pressure[0]) / 1e5
        
        # Power
        pump_power = value(pump.work_mechanical[0])
        
        # Ion concentrations and rejections
        ion_data = {}
        for comp in m.fs.properties.solute_set:
            feed_ion = value(ro.inlet.flow_mass_phase_comp[0, 'Liq', comp])
            perm_ion = value(ro.permeate.flow_mass_phase_comp[0, 'Liq', comp])
            conc_ion = value(ro.retentate.flow_mass_phase_comp[0, 'Liq', comp])
            
            # Calculate concentrations (mg/L)
            feed_conc = feed_ion / (feed_h2o + sum(
                value(ro.inlet.flow_mass_phase_comp[0, 'Liq', c]) 
                for c in m.fs.properties.solute_set
            )) * 1e6 if feed_h2o > 0 else 0
            
            perm_conc = perm_ion / (perm_h2o + sum(
                value(ro.permeate.flow_mass_phase_comp[0, 'Liq', c]) 
                for c in m.fs.properties.solute_set
            )) * 1e6 if perm_h2o > 0 else 0
            
            conc_conc = conc_ion / (conc_h2o + sum(
                value(ro.retentate.flow_mass_phase_comp[0, 'Liq', c]) 
                for c in m.fs.properties.solute_set
            )) * 1e6 if conc_h2o > 0 else 0
            
            # Rejection
            rejection = 1 - (perm_conc / feed_conc) if feed_conc > 0 else 0
            
            ion_data[comp] = {
                "feed_mg_l": feed_conc,
                "permeate_mg_l": perm_conc,
                "concentrate_mg_l": conc_conc,
                "rejection": rejection,
                "feed_flow_kg_s": feed_ion,
                "permeate_flow_kg_s": perm_ion,
                "concentrate_flow_kg_s": conc_ion
            }
        
        # Stage results
        stage_result = {
            "stage": i,
            "recovery": recovery,
            "feed_flow_kg_s": feed_h2o + sum(
                value(ro.inlet.flow_mass_phase_comp[0, 'Liq', c]) 
                for c in m.fs.properties.solute_set
            ),
            "permeate_flow_kg_s": perm_h2o + sum(
                value(ro.permeate.flow_mass_phase_comp[0, 'Liq', c]) 
                for c in m.fs.properties.solute_set
            ),
            "concentrate_flow_kg_s": conc_h2o + sum(
                value(ro.retentate.flow_mass_phase_comp[0, 'Liq', c]) 
                for c in m.fs.properties.solute_set
            ),
            "feed_pressure_bar": feed_pressure,
            "concentrate_pressure_bar": conc_pressure,
            "permeate_pressure_bar": perm_pressure,
            "pump_power_kW": abs(pump_power) / 1000,
            "ion_data": ion_data
        }
        
        results["stage_results"].append(stage_result)
        
        total_perm_flow += perm_h2o
        total_power += abs(pump_power)
    
    # Overall performance metrics
    if has_recycle:
        # For recycle systems, use fresh feed flow
        fresh_h2o = value(m.fs.fresh_feed.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        fresh_tds = sum(
            value(m.fs.fresh_feed.outlet.flow_mass_phase_comp[0, 'Liq', c])
            for c in m.fs.properties.solute_set
        )
        fresh_total = fresh_h2o + fresh_tds
        
        # Disposal flow (actual concentrate)
        disposal_h2o = value(m.fs.disposal_product.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        disposal_tds = sum(
            value(m.fs.disposal_product.inlet.flow_mass_phase_comp[0, 'Liq', c])
            for c in m.fs.properties.solute_set
        )
        
        system_recovery = total_perm_flow / fresh_h2o if fresh_h2o > 0 else 0
    else:
        # Standard system
        feed_h2o = value(m.fs.fresh_feed.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        feed_tds = sum(
            value(m.fs.fresh_feed.outlet.flow_mass_phase_comp[0, 'Liq', c])
            for c in m.fs.properties.solute_set
        )
        fresh_total = feed_h2o + feed_tds
        
        system_recovery = total_perm_flow / feed_h2o if feed_h2o > 0 else 0
    
    # Calculate overall TDS
    total_perm_tds = 0
    for i in range(1, n_stages + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        for comp in m.fs.properties.solute_set:
            total_perm_tds += value(ro.permeate.flow_mass_phase_comp[0, 'Liq', comp])
    
    perm_tds_mg_l = total_perm_tds / total_perm_flow * 1e6 if total_perm_flow > 0 else 0
    
    # Overall ion tracking
    overall_ions = {}
    for comp in m.fs.properties.solute_set:
        # Fresh feed
        fresh_ion = value(m.fs.fresh_feed.outlet.flow_mass_phase_comp[0, 'Liq', comp])
        fresh_conc = fresh_ion / fresh_total * 1e6 if fresh_total > 0 else 0
        
        # Total permeate
        perm_ion = sum(
            value(getattr(m.fs, f"ro_stage{i}").permeate.flow_mass_phase_comp[0, 'Liq', comp])
            for i in range(1, n_stages + 1)
        )
        perm_conc = perm_ion / (total_perm_flow + total_perm_tds) * 1e6 if total_perm_flow > 0 else 0
        
        # Disposal or concentrate
        if has_recycle:
            disposal_ion = value(m.fs.disposal_product.inlet.flow_mass_phase_comp[0, 'Liq', comp])
            disposal_conc = disposal_ion / (disposal_h2o + disposal_tds) * 1e6 if disposal_h2o > 0 else 0
        else:
            final_ro = getattr(m.fs, f"ro_stage{n_stages}")
            conc_ion = value(final_ro.retentate.flow_mass_phase_comp[0, 'Liq', comp])
            conc_h2o = value(final_ro.retentate.flow_mass_phase_comp[0, 'Liq', 'H2O'])
            conc_tds = sum(
                value(final_ro.retentate.flow_mass_phase_comp[0, 'Liq', c])
                for c in m.fs.properties.solute_set
            )
            disposal_conc = conc_ion / (conc_h2o + conc_tds) * 1e6 if conc_h2o > 0 else 0
        
        # Overall rejection
        overall_rejection = 1 - (perm_conc / fresh_conc) if fresh_conc > 0 else 0
        
        overall_ions[comp] = {
            "fresh_feed_mg_l": fresh_conc,
            "combined_permeate_mg_l": perm_conc,
            "disposal_mg_l": disposal_conc,
            "overall_rejection": overall_rejection
        }
    
    results["ion_tracking"] = overall_ions
    
    # Recycle-specific metrics
    if has_recycle:
        # Effective feed composition (after mixing with recycle)
        effective_ions = {}
        mixed_h2o = value(m.fs.feed_mixer.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O'])
        mixed_tds = sum(
            value(m.fs.feed_mixer.outlet.flow_mass_phase_comp[0, 'Liq', c])
            for c in m.fs.properties.solute_set
        )
        mixed_total = mixed_h2o + mixed_tds
        
        for comp in m.fs.properties.solute_set:
            mixed_ion = value(m.fs.feed_mixer.outlet.flow_mass_phase_comp[0, 'Liq', comp])
            mixed_conc = mixed_ion / mixed_total * 1e6 if mixed_total > 0 else 0
            
            # Accumulation factor
            fresh_conc = overall_ions[comp]["fresh_feed_mg_l"]
            accumulation = mixed_conc / fresh_conc if fresh_conc > 0 else 1
            
            effective_ions[comp] = {
                "effective_feed_mg_l": mixed_conc,
                "accumulation_factor": accumulation
            }
        
        results["recycle_metrics"] = {
            "recycle_split_ratio": value(m.fs.recycle_split.split_fraction[0, "recycle"]),
            "recycle_flow_kg_s": value(m.fs.recycle_split.recycle.flow_mass_phase_comp[0, 'Liq', 'H2O']),
            "disposal_flow_kg_s": disposal_h2o,
            "effective_ion_composition": effective_ions
        }
    
    # Performance summary
    results["performance"] = {
        "system_recovery": system_recovery,
        "total_permeate_flow_m3_h": total_perm_flow * 3.6,  # kg/s to m³/h
        "total_permeate_tds_mg_l": perm_tds_mg_l,
        "total_power_consumption_kW": total_power / 1000,
        "specific_energy_kWh_m3": (total_power / 1000) / (total_perm_flow * 3.6) if total_perm_flow > 0 else 0
    }
    
    # Mass balance check
    if has_recycle:
        # Fresh in = permeate out + disposal out
        fresh_in = fresh_h2o + fresh_tds
        perm_out = total_perm_flow + total_perm_tds
        disposal_out = disposal_h2o + disposal_tds
        
        mass_balance_error = abs(fresh_in - perm_out - disposal_out) / fresh_in if fresh_in > 0 else 0
    else:
        # Feed in = permeate out + concentrate out
        feed_in = fresh_total
        perm_out = total_perm_flow + total_perm_tds
        final_ro = getattr(m.fs, f"ro_stage{n_stages}")
        conc_out = sum(
            value(final_ro.retentate.flow_mass_phase_comp[0, 'Liq', c])
            for c in ['H2O'] + list(m.fs.properties.solute_set)
        )
        
        mass_balance_error = abs(feed_in - perm_out - conc_out) / feed_in if feed_in > 0 else 0
    
    results["mass_balance"] = {
        "mass_balance_error": mass_balance_error,
        "mass_balance_ok": mass_balance_error < 0.001
    }
    
    # Economics placeholder
    results["economics"] = {
        "capital_cost_usd": 0,  # Would need costing correlations
        "operating_cost_usd_year": 0,
        "lcow_usd_m3": 0
    }
    
    return results

## Run RO Simulation with MCAS and Recycle

In [ ]:
try:
    # Build the model
    print("Building RO model with MCAS property package...")
    if has_recycle:
        print("Including recycle system components...")
    
    model = build_ro_model_mcas_with_recycle(
        configuration, 
        mcas_config, 
        feed_salinity_ppm,
        feed_temperature_c, 
        membrane_type,
        has_recycle
    )
    
    print("\nModel structure created successfully")
    print(f"Degrees of freedom: {degrees_of_freedom(model)}")
    
    # Initialize and solve
    print("\nInitializing and solving model...")
    print(f"Pump optimization: {'ENABLED' if optimize_pumps else 'DISABLED'}")
    
    model = initialize_and_solve_mcas(model, configuration, optimize_pumps)
    
    print("\nModel solved successfully!")
    
    # Extract results
    print("\nExtracting results...")
    results = extract_results_mcas(model, configuration)
    
    print("\nSimulation completed successfully!")
    
except Exception as e:
    import traceback
    print(f"\nERROR: {str(e)}")
    print("\nFull traceback:")
    traceback.print_exc()
    
    results = {
        "status": "error",
        "message": str(e),
        "traceback": traceback.format_exc(),
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {}
    }

## Display Results

In [ ]:
if results["status"] == "success":
    print("=== SIMULATION RESULTS ===\n")
    
    # Overall performance
    perf = results["performance"]
    print("Overall System Performance:")
    print(f"  System Recovery: {perf['system_recovery']:.1%}")
    print(f"  Permeate Flow: {perf['total_permeate_flow_m3_h']:.1f} m³/h")
    print(f"  Permeate TDS: {perf['total_permeate_tds_mg_l']:.0f} mg/L")
    print(f"  Total Power: {perf['total_power_consumption_kW']:.1f} kW")
    print(f"  Specific Energy: {perf['specific_energy_kWh_m3']:.2f} kWh/m³")
    
    # Stage results
    print("\nStage-wise Results:")
    print("Stage | Recovery | Feed P  | Perm Flow | Power")
    print("      |    (%)   | (bar)   | (m³/h)    | (kW)")
    print("-" * 50)
    
    for stage in results["stage_results"]:
        print(f"  {stage['stage']}   | {stage['recovery']*100:6.1f}  | "
              f"{stage['feed_pressure_bar']:7.1f} | "
              f"{stage['permeate_flow_kg_s']*3.6:9.1f} | "
              f"{stage['pump_power_kW']:6.1f}")
    
    # Ion tracking
    print("\nOverall Ion Rejection:")
    print("Ion     | Feed    | Permeate | Rejection")
    print("        | (mg/L)  | (mg/L)   | (%)")
    print("-" * 45)
    
    for ion, data in sorted(results["ion_tracking"].items()):
        print(f"{ion:8s}| {data['fresh_feed_mg_l']:7.1f} | "
              f"{data['combined_permeate_mg_l']:8.2f} | "
              f"{data['overall_rejection']*100:6.1f}")
    
    # Recycle metrics if applicable
    if has_recycle and "recycle_metrics" in results:
        rm = results["recycle_metrics"]
        print("\nRecycle System Metrics:")
        print(f"  Recycle Split Ratio: {rm['recycle_split_ratio']*100:.1f}%")
        print(f"  Recycle Flow: {rm['recycle_flow_kg_s']*3.6:.1f} m³/h")
        print(f"  Disposal Flow: {rm['disposal_flow_kg_s']*3.6:.1f} m³/h")
        
        print("\nIon Accumulation Factors:")
        for ion, data in sorted(rm["effective_ion_composition"].items()):
            print(f"  {ion}: {data['accumulation_factor']:.2f}x "
                  f"(effective: {data['effective_feed_mg_l']:.0f} mg/L)")
    
    # Mass balance
    mb = results["mass_balance"]
    print(f"\nMass Balance Error: {mb['mass_balance_error']*100:.3f}%")
    print(f"Mass Balance Check: {'PASS' if mb['mass_balance_ok'] else 'FAIL'}")
    
else:
    print("=== SIMULATION ERROR ===")
    print(f"Error: {results['message']}")
    if 'traceback' in results:
        print("\nTraceback:")
        print(results['traceback'])

# Return results for papermill extraction

In [ ]:
# Return results as JSON for papermill
import json
json.dumps(results)